## 1 - Creating the Blackjack Game Logic

In this notebook I go over my class implementations for the setup of the Blackjack game. I opted for a simple OOP-based structure that would allow for easier replicability to real life, by creating Card, Hand, and Decks classes. For a more in-depth look at the code, check out [blackjack_utils](../src/blackjack_utils.py).

As a simple explanation of the version of Blackjack I'm using, a player has 4 choices on their first deal of cards. The first 2 are hitting, which will draw a card, and standing, which means the player is satisfied with their current total. Otherwise, the player can double down, which will force them to draw exactly 1 more card while doubling their bet. The final option is to split, which--in the case of a pair being dealt to the player--allows them to break their hand into 2 separate hands with each hand matching the bet amount on the initial hand. A more detailed description of the exact rules being used is in the simulation section below.

In [1]:
import sys
sys.path.append('../src')

from blackjack_utils import Card, Hand, Decks, split_hands, dealer_simulation, simulation

### Card Class

The first part of building a Blackjack game is to create the objects used in the game, which in our situation are playing cards. Each card has an assigned rank, suit, and value at initialization as displayed below. I also defined an equals method in order to compare whether two cards had the same value, and this was useful for the case of splitting pairs.

In [5]:
help(Card)

Help on class Card in module blackjack_utils:

class Card(builtins.object)
 |  Card(rank, suit, value)
 |
 |  # Playing card class with each card having rank, suit, value (e.g. Jack, Diamond, 10)
 |
 |  Methods defined here:
 |
 |  __eq__(self, other)
 |      Return self==value.
 |
 |  __init__(self, rank, suit, value)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object
 |
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |
 |  __hash__ = None



### Hand Class

The next class I created was a Hand class, in order to simulate the cards held by both the player and the dealer. Each hand contains a list in order to hold the cards, as well as methods to easily index a specific card as well as return how many cards are in the hand. Additionally, due to the nuances of aces in Blackjack, I opted to declare an ace's value to be 11 unless the hand total would go over 21. Hence, the class contains 2 additional variables for the purposes of counting the number of 11-valued aces as well as the current card total. Lastly, each hand also has a split method in order to simplify the process of splitting a pair into 2 separate hands.

In [6]:
help(Hand)

Help on class Hand in module blackjack_utils:

class Hand(builtins.object)
 |  # Hand assumes that ace is 11 until it would cause player to bust, then it
 |  # simulates changing the value of ace to 1
 |
 |  Methods defined here:
 |
 |  __getitem__(self, card_number)
 |
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __len__(self)
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  has_eleven_ace(self)
 |
 |  receive_card(self, new_card)
 |
 |  split(self, other)
 |
 |  value(self)
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



### Decks Class

The next class I defined was a Decks class. The reason for the plurality is due to the fact that in Blackjack, it is common practice for casinos to use multiple decks all shuffled together, which are held in a container known as a "shoe." This class initializes a set number of decks (taken as input) and contains methods to shuffle the decks as well as deal cards to the player's or dealer's hands.

In [10]:
help(Decks)

Help on class Decks in module blackjack_utils:

class Decks(builtins.object)
 |  Decks(num_decks=1)
 |
 |  # This will be used to simulate the shoe of cards, so we can make shoes
 |  # by picking any arbitrary number of decks
 |
 |  Methods defined here:
 |
 |  __init__(self, num_decks=1)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  __len__(self)
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  deal(self, hand)
 |
 |  shuffle(self)
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object



### Hand Splitting Function

In order to facilitate hand splitting, I created a method to break apart a player's hand into 2 separate hands and deal a card to each and make them proper hands.

In [34]:
player_hands = [Hand()]
cards = Decks(2)
cards.shuffle()

for _ in range(2):
    cards.deal(player_hands[0])

print('The', player_hands[0])

The hand is worth 6 and contains:
3 of Hearts
3 of Hearts


In [35]:
split_hands(player_hands, cards)

for hand in player_hands:
    print('The', hand, '\n')

The hand is worth 9 and contains:
3 of Hearts
6 of Hearts 

The hand is worth 13 and contains:
3 of Hearts
Jack of Hearts 



### Dealer Simulation Function

This is the first step of actual game logic. The dealer has a set rulebook for every turn and can't decide what they want to do with their cards. In this version, I chose to make the dealer play by Soft 17 rules. This means that for every hand total less than or equal to 16, the dealer HAS to hit, and for every hand total greater than or equal to 17, the dealer HAS to stand. The reason for Soft 17 rules, however, is that when the dealer has an ace which is considered to be "11-valued", they can always still hit without any risk of busting, as they can simply turn the ace into a card with 1 value. This means that anytime the dealer reaches a hand total of 17 with a "soft ace" (having 11 value) they will be forced to hit, and in every other situation they must stand at 17 or greater.

In [37]:
dealer_hand = Hand()

for _ in range(2):
    cards.deal(dealer_hand)

print ('The dealer', dealer_hand)

The dealer hand is worth 10 and contains:
7 of Diamonds
3 of Diamonds


In [38]:
dealer_simulation(dealer_hand, cards)

print ('The dealer', dealer_hand)

The dealer hand is worth 22 and contains:
7 of Diamonds
3 of Diamonds
4 of Clubs
8 of Hearts


In the above situation, the dealer busted as their card total went over 21 after they were forced to hit at a total of 14 (7 + 3 + 4).

### Main Simulation Function

The `simulation()` function is the main method used to test out different Blackjack strategies. It accepts 5 parameters: `strategy`, `initial_money`, `bet_amount`, `max_rounds`, and `decks`. The `initial_money` and `bet_amount` decide the initial amount brought by the player and the amount bet every round. The `max_rounds` and `decks` dictate how many rounds of the game will be played and the number of decks to be used. The `strategy` parameter is the most important one, as it controls the flow of the game for the player. It takes a function as an input and plays out the player's turn, only returning whether the player chose to double down or not. The `strategy` function takes the player's hand, the deck of cards, the dealer's up card, and whether the player can bet more as its 4 inputs.

Using all the above parameters, the `simulation()` function plays out a full game of Blackjack until either the number of specified rounds are over or the player has lost all their money, and it returns the total gain/loss since the player started playing.

In [2]:
help(simulation)

Help on function simulation in module blackjack_utils:

simulation(strategy, initial_money=1000, bet_amount=100, max_rounds=1, decks=6)
    # Blackjack Configuration
    # - Cards reshuffled after 25% deck penetration
    # - 1:1 payout
    # - Dealer hits soft 17
    # - Player actions: (H)it, (S)tand, s(P)lit, (D)ouble down
    # - No resplits
    # - No double down after split
    # - No surrender
    # Customizable Configuration
    # - Player strategy
    # - Initial money (Default: 1000)
    # - Bet amount (Default: 100)
    # - Max rounds played (Default: 1)
    # - Number of decks used for the shoe (Default: 6)



The exact rules for my version of Blackjack are listed above. In more detail, the rules are as follows:

1) The shoe of cards will be reshuffled after the deck penetration reaches 25%, which means that 75% of the cards have been used.
2) The player will be paid out 1:1 on all Blackjacks, so a winning hand with a 21 value will only receive the bet amount.
3) The dealer follows soft 17 rules and must hit if they have a soft ace with a total of 17.
4) The player has 4 actions, which are hitting, standing splitting, and doubling down.
5) Resplits and doubling down after splitting are not allowed, so the only turn where splitting or doubling down are possible are on the first deal.
6) There is no insurance or surrendering.